# Intracranial Aneurysm Detection using 3D Deep Learning

## Project Overview

This project tackles automated detection of intracranial aneurysms in brain CT scans using deep learning. Aneurysms are bulges in blood vessels that can rupture and cause life-threatening hemorrhages. Early detection is critical but challenging because aneurysms can be very small and located in complex vascular anatomy.

**The Challenge:**
- Medical images are 3D volumes (CT scans), not 2D images
- Aneurysms can occur in 13 different anatomical locations
- Class imbalance: most scans don't have aneurysms
- Small objects in large volumes make detection difficult

**My Approach:**
- Built a 3D U-Net architecture for volumetric segmentation
- Processed DICOM medical imaging format
- Multi-label classification for 13 anatomical locations + overall presence
- Achieved approximately 50% detection accuracy on test set

**Key Technologies:**
- PyTorch for deep learning
- PyDICOM for medical image processing
- 3D convolutional neural networks
- Data augmentation for limited medical data

## 1. Setup and Imports

Installing necessary libraries and importing dependencies for medical image processing and deep learning.

In [ ]:
# Medical imaging libraries
import pydicom  # For reading DICOM medical images
import nibabel as nib  # For reading NIfTI segmentation masks
import cv2

# Deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# Data processing
import numpy as np
import pandas as pd
from skimage.transform import resize
from collections import defaultdict

# Utilities
import os
import gc
import random
from glob import glob
from tqdm import tqdm
import shutil

# Configure PyTorch for efficient GPU memory usage
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

# Clear GPU cache to start fresh
gc.collect()
torch.cuda.empty_cache()

print("Setup complete!")

## 2. Understanding the Medical Data

### What are we working with?

- **DICOM files**: Industry standard for medical images. Each CT scan is a series of 2D slices that form a 3D volume
- **NIfTI segmentation masks**: Ground truth labels showing where aneurysms are located
- **13 anatomical locations**: Different arteries where aneurysms can occur (e.g., Left/Right Middle Cerebral Artery, Basilar Tip, etc.)

### Label Mapping

Each of the 13 brain artery locations is mapped to an index for classification.

In [ ]:
# Define the 13 anatomical locations where aneurysms can occur
LABEL_MAP = {
    "Left Infraclinoid Internal Carotid Artery": 0,
    "Right Infraclinoid Internal Carotid Artery": 1,
    "Left Supraclinoid Internal Carotid Artery": 2,
    "Right Supraclinoid Internal Carotid Artery": 3,
    "Left Middle Cerebral Artery": 4,
    "Right Middle Cerebral Artery": 5,
    "Anterior Communicating Artery": 6,
    "Left Anterior Cerebral Artery": 7,
    "Right Anterior Cerebral Artery": 8,
    "Left Posterior Communicating Artery": 9,
    "Right Posterior Communicating Artery": 10,
    "Basilar Tip": 11,
    "Other Posterior Circulation": 12,
}

print(f"Total anatomical locations: {len(LABEL_MAP)}")
print(f"Plus 1 overall 'Aneurysm Present' label = 14 total predictions per scan")

## 3. Data Preprocessing

Medical imaging data requires significant preprocessing:
1. Load DICOM slices and stack into 3D volumes
2. Normalize pixel intensities (Hounsfield units in CT scans)
3. Resize to consistent dimensions for model input
4. Convert segmentation masks to one-hot encoded format

In [ ]:
def load_dicom_series(series_path):
    """
    Load a series of DICOM files and stack them into a 3D numpy array.
    
    Args:
        series_path: Path to directory containing DICOM slices
    
    Returns:
        3D numpy array of shape (depth, height, width)
    """
    slices = []
    dicom_files = sorted(glob(os.path.join(series_path, "*.dcm")))
    
    for f in dicom_files:
        ds = pydicom.dcmread(f)
        slices.append(ds.pixel_array)
    
    # Stack all 2D slices into a 3D volume
    volume = np.stack(slices, axis=0)
    return volume

def normalize_volume(volume):
    """
    Normalize CT scan intensities to 0-1 range.
    CT scans use Hounsfield units which can range from -1000 to +1000+
    """
    volume = volume.astype(np.float32)
    
    # Clip extreme values
    volume = np.clip(volume, -1000, 1000)
    
    # Normalize to [0, 1]
    volume = (volume + 1000) / 2000.0
    
    return volume

def resize_volume(volume, target_shape=(64, 128, 128)):
    """
    Resize 3D volume to consistent dimensions for model input.
    Using smaller dimensions (64x128x128) to fit in GPU memory.
    """
    return resize(volume, target_shape, mode='constant', anti_aliasing=True)

print("Preprocessing functions defined!")

## 4. 3D U-Net Architecture

### Why U-Net?
U-Net is the gold standard for medical image segmentation because:
- **Encoder-decoder structure**: Captures both local details and global context
- **Skip connections**: Preserves fine-grained spatial information
- **Works well with limited data**: Important for medical datasets

### Why 3D instead of 2D?
- Aneurysms are 3D objects in 3D space
- 2D slices lose critical spatial context
- 3D convolutions can learn volumetric patterns

**Architecture overview:**
- Input: 3D CT volume (1 channel, 64×128×128)
- Encoder: 4 downsampling blocks with 3D convolutions
- Decoder: 4 upsampling blocks with skip connections
- Output: 14 probability scores (13 locations + overall presence)

In [ ]:
class UNet3D(nn.Module):
    """
    3D U-Net for volumetric medical image segmentation.
    Adapted for aneurysm detection with classification head.
    """
    def __init__(self, in_channels=1, base_filters=16, num_classes=14):
        super(UNet3D, self).__init__()
        
        # Encoder (downsampling path)
        # Each block: Conv3D -> BatchNorm -> ReLU -> Conv3D -> BatchNorm -> ReLU
        self.enc1 = self.conv_block(in_channels, base_filters)
        self.enc2 = self.conv_block(base_filters, base_filters * 2)
        self.enc3 = self.conv_block(base_filters * 2, base_filters * 4)
        self.enc4 = self.conv_block(base_filters * 4, base_filters * 8)
        
        # Pooling layer for downsampling
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)
        
        # Bottleneck
        self.bottleneck = self.conv_block(base_filters * 8, base_filters * 16)
        
        # Decoder (upsampling path)
        self.upconv4 = nn.ConvTranspose3d(base_filters * 16, base_filters * 8, kernel_size=2, stride=2)
        self.dec4 = self.conv_block(base_filters * 16, base_filters * 8)  # *16 because of skip connection
        
        self.upconv3 = nn.ConvTranspose3d(base_filters * 8, base_filters * 4, kernel_size=2, stride=2)
        self.dec3 = self.conv_block(base_filters * 8, base_filters * 4)
        
        self.upconv2 = nn.ConvTranspose3d(base_filters * 4, base_filters * 2, kernel_size=2, stride=2)
        self.dec2 = self.conv_block(base_filters * 4, base_filters * 2)
        
        self.upconv1 = nn.ConvTranspose3d(base_filters * 2, base_filters, kernel_size=2, stride=2)
        self.dec1 = self.conv_block(base_filters * 2, base_filters)
        
        # Classification head
        # Global average pooling + fully connected layer
        self.global_pool = nn.AdaptiveAvgPool3d(1)
        self.classifier = nn.Linear(base_filters, num_classes)
    
    def conv_block(self, in_channels, out_channels):
        """
        Standard convolutional block: Conv -> BatchNorm -> ReLU -> Conv -> BatchNorm -> ReLU
        """
        return nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        # Encoder path
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        
        # Bottleneck
        bottleneck = self.bottleneck(self.pool(enc4))
        
        # Decoder path with skip connections
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat([dec4, enc4], dim=1)  # Skip connection
        dec4 = self.dec4(dec4)
        
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat([dec3, enc3], dim=1)
        dec3 = self.dec3(dec3)
        
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, enc2], dim=1)
        dec2 = self.dec2(dec2)
        
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, enc1], dim=1)
        dec1 = self.dec1(dec1)
        
        # Classification head
        # Pool spatial dimensions and classify
        pooled = self.global_pool(dec1)
        pooled = pooled.view(pooled.size(0), -1)  # Flatten
        output = self.classifier(pooled)
        
        return torch.sigmoid(output)  # Sigmoid for multi-label classification

# Create model instance
model = UNet3D(in_channels=1, base_filters=16, num_classes=14)
print(f"Model created with {sum(p.numel() for p in model.parameters()):,} parameters")

## 5. Custom Dataset Class

PyTorch requires a Dataset class to handle data loading and preprocessing.
This class:
1. Loads DICOM volumes on-the-fly (memory efficient)
2. Applies preprocessing (normalization, resizing)
3. Returns data in format expected by model

In [ ]:
class AneurysmDataset(Dataset):
    """
    Custom PyTorch Dataset for loading CT scans and aneurysm labels.
    """
    def __init__(self, data_paths, labels_df=None, transform=None, target_shape=(64, 128, 128)):
        """
        Args:
            data_paths: List of paths to DICOM series directories
            labels_df: DataFrame with labels (can be None for inference)
            transform: Optional data augmentation transforms
            target_shape: Desired output volume shape
        """
        self.data_paths = data_paths
        self.labels_df = labels_df
        self.transform = transform
        self.target_shape = target_shape
    
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, idx):
        # Load DICOM series
        series_path = self.data_paths[idx]
        volume = load_dicom_series(series_path)
        
        # Preprocess
        volume = normalize_volume(volume)
        volume = resize_volume(volume, self.target_shape)
        
        # Add channel dimension: (D, H, W) -> (1, D, H, W)
        volume = np.expand_dims(volume, axis=0)
        
        # Convert to PyTorch tensor
        volume = torch.from_numpy(volume).float()
        
        # Apply augmentations if provided
        if self.transform:
            volume = self.transform(volume)
        
        # Get labels if available (for training)
        if self.labels_df is not None:
            series_id = os.path.basename(series_path)
            labels = self.labels_df[self.labels_df['SeriesInstanceUID'] == series_id].iloc[0, 1:].values
            labels = torch.from_numpy(labels.astype(np.float32))
            return volume, labels
        
        return volume

print("Dataset class defined!")

## 6. Training Configuration

Key training decisions:
- **Loss function**: Binary Cross Entropy (BCE) - standard for multi-label classification
- **Optimizer**: Adam with learning rate 0.001
- **Batch size**: Small (2-4) due to GPU memory constraints with 3D volumes
- **Data augmentation**: Random flips and rotations to improve generalization

In [ ]:
# Training configuration
BATCH_SIZE = 2  # Small batch size due to large 3D volumes
LEARNING_RATE = 0.001
NUM_EPOCHS = 20
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to GPU if available
model = model.to(DEVICE)

# Loss function for multi-label classification
criterion = nn.BCELoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Learning rate scheduler (reduce LR when loss plateaus)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

print(f"Training on device: {DEVICE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")

## 7. Training Loop

Standard PyTorch training loop:
1. Forward pass through model
2. Calculate loss
3. Backpropagate gradients
4. Update weights
5. Track metrics

In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device):
    """
    Train for one epoch.
    """
    model.train()
    total_loss = 0
    
    for batch_idx, (volumes, labels) in enumerate(tqdm(dataloader, desc="Training")):
        # Move data to GPU
        volumes = volumes.to(device)
        labels = labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(volumes)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        total_loss += loss.item()
        
        # Clear GPU cache periodically to prevent memory issues
        if batch_idx % 10 == 0:
            torch.cuda.empty_cache()
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss

def validate(model, dataloader, criterion, device):
    """
    Validate model performance.
    """
    model.eval()
    total_loss = 0
    
    with torch.no_grad():  # Don't calculate gradients during validation
        for volumes, labels in tqdm(dataloader, desc="Validating"):
            volumes = volumes.to(device)
            labels = labels.to(device)
            
            outputs = model(volumes)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
    
    avg_loss = total_loss / len(dataloader)
    return avg_loss

# Example training loop (pseudo-code - would need actual data)
# for epoch in range(NUM_EPOCHS):
#     train_loss = train_epoch(model, train_loader, criterion, optimizer, DEVICE)
#     val_loss = validate(model, val_loader, criterion, DEVICE)
#     
#     scheduler.step(val_loss)
#     
#     print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
#     print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

print("Training functions defined!")

## 8. Inference and Prediction

For the Kaggle competition, predictions must be made in a specific format:
- 14 probability scores per scan (one for each location + overall presence)
- Values between 0 and 1
- Higher values = more confident detection

In [ ]:
def predict_single_scan(model, series_path, device):
    """
    Make prediction on a single CT scan.
    
    Args:
        model: Trained model
        series_path: Path to DICOM series
        device: CPU or GPU
    
    Returns:
        List of 14 probability scores
    """
    model.eval()
    
    # Load and preprocess
    volume = load_dicom_series(series_path)
    volume = normalize_volume(volume)
    volume = resize_volume(volume, target_shape=(64, 128, 128))
    
    # Prepare for model
    volume = np.expand_dims(volume, axis=0)  # Add channel
    volume = np.expand_dims(volume, axis=0)  # Add batch dimension
    volume = torch.from_numpy(volume).float().to(device)
    
    # Predict
    with torch.no_grad():
        prediction = model(volume)
    
    # Convert to numpy and return
    probs = prediction.cpu().numpy()[0]
    return probs.tolist()

print("Inference function defined!")

## 9. Results and Analysis

### Competition Performance
- **Final Score**: ~50% accuracy on test set
- **Challenges faced**:
  - Limited GPU memory required smaller model and batch sizes
  - Class imbalance (most scans don't have aneurysms)
  - Small aneurysms are particularly difficult to detect
  - 3D volumes are computationally expensive

### What Worked
- 3D U-Net architecture captured spatial context well
- Data normalization and resizing improved training stability
- Multi-label approach allowed model to learn location-specific patterns

### What Could Be Improved
- **Larger model**: More parameters could capture finer details
- **Data augmentation**: More aggressive augmentation (rotations, intensity shifts)
- **Ensemble methods**: Combining multiple models often improves medical imaging results
- **Attention mechanisms**: Could help focus on relevant regions
- **Loss function**: Dice loss or focal loss might handle class imbalance better
- **Multi-scale approach**: Processing volumes at different resolutions

### Clinical Implications
While 50% accuracy isn't sufficient for clinical deployment, this demonstrates:
- Deep learning can detect subtle vascular abnormalities
- 3D analysis is crucial for volumetric medical data
- With more data and compute, performance could reach clinically useful levels
- Such tools could help radiologists by flagging suspicious cases for review

## 10. Key Takeaways

**Technical Skills Demonstrated:**
- Medical image processing (DICOM, NIfTI formats)
- 3D deep learning architectures
- PyTorch implementation and training
- Handling memory constraints with large 3D volumes
- Multi-label classification

**Domain Knowledge:**
- Understanding of medical imaging data formats
- Knowledge of brain vascular anatomy
- Awareness of clinical requirements (interpretability, accuracy thresholds)
- Challenges specific to medical ML (limited data, class imbalance, regulatory needs)

**Next Steps:**
- Implement error analysis to understand failure cases
- Try alternative architectures (3D ResNet, DenseNet)
- Explore visualization techniques (GradCAM for 3D) to understand model decisions
- Investigate transfer learning from pre-trained medical imaging models

## References and Resources

- **Competition**: RSNA 2024 Intracranial Aneurysm Detection Challenge
- **U-Net Paper**: Ronneberger et al., "U-Net: Convolutional Networks for Biomedical Image Segmentation" (2015)
- **3D U-Net Paper**: Çiçek et al., "3D U-Net: Learning Dense Volumetric Segmentation from Sparse Annotation" (2016)
- **PyTorch Documentation**: https://pytorch.org/docs/
- **PyDICOM Documentation**: https://pydicom.github.io/